### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
# %pip install --upgrade transformers datasets accelerate deepspeed

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score
device = 'cuda'

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Load data and model

In [3]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Repo card metadata block was not found. Setting CardData to empty.




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [70]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [71]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [72]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [73]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [76]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [77]:
model = model.to(device)

In [78]:
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=16, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=6
)
preds = []
gts = []
for i, batch in enumerate(tqdm(val_loader)):
    batch = {k: v.to(device) for k, v in batch.items()}

    predicted = model(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        token_type_ids=batch['token_type_ids']
    )
    preds.extend(torch.argmax(predicted.logits, dim=-1).data.cpu().numpy())
    gts.extend(batch['labels'].data.cpu().numpy())
accuracy = accuracy_score(gts, preds)
print(f'Accuracy is {accuracy}')

100%|██████████| 2527/2527 [02:14<00:00, 18.80it/s]

Accuracy is 0.9083848627256987


In [79]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [4]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

In [5]:
model_name = "microsoft/deberta-v3-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, output_hidden_states=True)

/home/misha/miniconda/envs/ml-311/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'classifier.weight', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [7]:
val_set = qqp_preprocessed['validation']
train_set = qqp_preprocessed['train']

In [8]:
model = model.to(device)

In [9]:
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=8, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=6
)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=8, shuffle=True, collate_fn=transformers.default_data_collator, num_workers=6
)

In [10]:
def validation_one_epoch(val_loader, model):
    preds = []
    gts = []
    model.eval()
    for i, batch in enumerate(tqdm(val_loader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            predicted = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                token_type_ids=batch['token_type_ids']
            )
            preds.extend(torch.argmax(predicted.logits, dim=-1).data.cpu().numpy())
            gts.extend(batch['labels'].data.cpu().numpy())
    accuracy = accuracy_score(gts, preds)
    return accuracy

def train_one_epoch(train_loader, model, criterion, opt):
    epoch_losses = []
    model.train(True)
    for i, batch in enumerate(tqdm(train_loader)):
        opt.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}

        predicted = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids']
        )
        loss = criterion(predicted.logits, batch['labels'])
        loss.backward()
        opt.step()
        if i % 1000 == 0:
            print(f'{loss.item()}')
        epoch_losses.append(loss.item())
    return epoch_losses


In [11]:
starting_accuracy = validation_one_epoch(val_loader, model)
print(f'Starting accuracy is {starting_accuracy}')


100%|██████████| 5054/5054 [03:15<00:00, 25.84it/s]

Starting accuracy is 0.60563937670047


In [14]:
opt = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()
num_epoch = 2

In [13]:
train_losses = []
val_accuracies = []
for epoch in range(num_epoch):
    print(f'Epoch {epoch} started')
    epoch_losses = train_one_epoch(train_loader, model, criterion, opt)
    train_losses.extend(epoch_losses)

    epoch_accuracy = validation_one_epoch(val_loader, model)
    val_accuracies.append(epoch_accuracy)
    print(f'Epoch train loss: {np.mean(epoch_losses)}, Epoch accuracy: {epoch_accuracy}')

Epoch 0 started


  0%|          | 2/45481 [00:00<2:52:14,  4.40it/s]

0.6778844594955444


  2%|▏         | 1002/45481 [02:34<1:54:17,  6.49it/s]

0.250326931476593


...

0.24287566542625427


 99%|█████████▉| 45002/45481 [1:55:53<01:14,  6.47it/s]

0.2909305691719055


100%|██████████| 5054/5054 [03:18<00:00, 25.47it/s]

Epoch train loss: 0.17162563806994696, Epoch accuracy: 0.9194410091516201


In [14]:
# torch.save(model.state_dict(), 'deberta_2_epochs.pt')

In [15]:
model.load_state_dict(torch.load('deberta_2_epochs.pt'))

<All keys matched successfully>

In [16]:
final_accuracy = validation_one_epoch(val_loader, model)
print(f'Final accuracy is {final_accuracy}')

100%|██████████| 5054/5054 [03:16<00:00, 25.68it/s]

Final accuracy is 0.9194410091516201


### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [50]:
MAX_LENGTH = 128
def preprocess_function_single(examples):
    del examples['text2']
    result_1 = tokenizer(
        examples['text1'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    return result_1

qqp_preprocessed_single = qqp.map(preprocess_function_single, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [51]:
train_set_single = qqp_preprocessed_single['train']

In [56]:
non_shuffled_train_loader = torch.utils.data.DataLoader(
    train_set_single, batch_size=8, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=6
)

In [57]:
def get_phrase_embedding(question, model, tokenizer):
    batch = tokenizer(question, padding='max_length', max_length=MAX_LENGTH, truncation=True)
    batch = {k: torch.as_tensor([v]).to(device) for k, v in batch.items() if k!='idx'}
    predicted = model(**batch)
    embs = predicted['hidden_states'][-1][:, 1:, :]
    all_embs = embs.cpu().detach().numpy().sum(axis=1)
    vector = all_embs[0]
    return vector

In [58]:
def find_nearest_nearpy(question, model, tokenizer):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_emb = get_phrase_embedding(question, model, tokenizer)
    top_k_indexes = np.array([int(x[1].split('_')[1]) for x in engine.neighbours(query_emb)])
    top_k_neighbours = train_set_single[top_k_indexes]
    return {k: v for k, v in top_k_neighbours.items() if k in['text1', 'text2']}

In [59]:
import numpy
# import tqdm

from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections

dimension = 768

rbp = RandomBinaryProjections('rbp', 10)

engine = Engine(dimension, lshashes=[rbp])

with torch.no_grad():
    model.eval()
    for index, batch in enumerate(tqdm(non_shuffled_train_loader)):
        batch = {k: v.to(device) for k, v in batch.items() if k!='idx'}
        predicted = model(**batch)
        embs = predicted['hidden_states'][-1][:, 1:, :]
        n = embs.shape[0]

        all_embs = embs.cpu().numpy().sum(axis=1)
        for i in range(n):
            engine.store_vector(all_embs[i], 'data_%d' % (index*len(batch) + i))

100%|██████████| 45481/45481 [30:15<00:00, 25.05it/s]


In [64]:
q_r = find_nearest_nearpy('What is it like to be in prison?', model, tokenizer)
q_r

{'text1': ['What is the best way to take a picture with a phone?',
  'Do u believe in aliens?',
  'What is the best gay Asian dating sites or apps?',
  'Which is the best campus ambassador program running?',
  'What will be the effect of banning 500 and 1000 notes on stock markets in India?',
  'Can we download movies from torrents?',
  'What heart rate should I target for in a 5 K?',
  'Can a poor man fall in love with a girl from a rich family?',
  'How is Kafka different from typical JMS message brokers like IBM MQ, Active MQ, etc.?',
  'What are the best and worst things about public transit in Enping, Guangdong, China? How could it be improved?']}

In [66]:
q_r = find_nearest_nearpy('What is the mass of the Earth?', model, tokenizer)
q_r

{'text1': ['I am an NRI and recently purchased a flat in India and other assets … tax form required?',
  'How long does 5 mg of Klonopin stay in your system?',
  'How do I perform meditation techniques such as the Sudarshan Kriya, Vipassana, Isha Kriya, and Transcendental meditation?',
  'How do I start from the scraps in an online business?',
  'What are your three wishes?',
  'How do I get into University of Pennsylvania Jerome Fisher M&T Program?',
  "Why don't people learn from their mistakes?",
  "What's new about iOS 10?",
  'What are the best qualities of an engineer?',
  'Is the world getting worse?']}

### It looks like I made a mistake somewhere, because neighbours are not good at all.
What I did: took all tokens from last hidden state, summed them together and then used the ANN engine to search for them.

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.